In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.utils import save_image
from torchvision.io import read_image
from torch.utils.data import DataLoader,Dataset, ConcatDataset
# import torchsummary
import torchattacks
from autoattack import AutoAttack
import timm
 
import time
import os
import copy
import robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from scripts.custom_Dataset import CustomDataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),])
                                # transforms.Resize((96,96)),
                    #   normalize])
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
batch_size = 2048
# num_classes = 1000

imagenet_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
test_loader = DataLoader(torchvision.datasets.ImageNet(root="../data/", split="val", transform=imagenet_transform),batch_size=batch_size,num_workers=8)

In [3]:
# image_folder = '../BLIP/experiment_1/imagenet/clean_test_2048/'
# label_file = '../BLIP/experiment_1/imagenet/clean_test_labels_2048.txt'
image_folder = '../BLIP/experiment_1/cifar10/clean_test/'
label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
# # captions_file = "../BLIP/experiment_1/stl_10/clean_test_captions.txt"
test_dataset = CustomDataset(image_folder, label_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


In [4]:
# model = models.resnet50(pretrained=True)
# model = load_model(model_name='Standard', dataset='cifar10', threat_model='Linf').eval()
# model = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf').eval()
model = torch.load("wrn28_10_cifar10_trained_on_diffusion_images_only.pth").module.eval()
# model = timm.create_model('wide_resnet50_2', pretrained=True).cuda().eval()
model = nn.DataParallel(model).to(device).eval()

In [5]:

images_list = []
labels_list = []

for images, labels in test_loader:
    images_list.append(images)
    labels_list.append(labels)

# Concatenate all batches
x = torch.cat(images_list, 0)
y = torch.cat(labels_list, 0)


In [6]:
len(x)

50000

In [7]:
# adversary = AutoAttack(model, norm='Linf', eps=4/255, version='standard', attacks_to_run=['apgd-ce', 'apgd-dlr'], device=device)
adversary = AutoAttack(model, norm="Linf", eps=4/255, device=device)
# adversary.apgd.n_restarts = 1

setting parameters for standard version


In [8]:
x_adv = adversary.run_standard_evaluation(x, y, 512)

using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 80.00%
apgd-ce - 1/79 - 512 out of 512 successfully perturbed
apgd-ce - 2/79 - 512 out of 512 successfully perturbed
apgd-ce - 3/79 - 512 out of 512 successfully perturbed
apgd-ce - 4/79 - 512 out of 512 successfully perturbed
apgd-ce - 5/79 - 512 out of 512 successfully perturbed
apgd-ce - 6/79 - 512 out of 512 successfully perturbed
apgd-ce - 7/79 - 512 out of 512 successfully perturbed
apgd-ce - 8/79 - 512 out of 512 successfully perturbed
apgd-ce - 9/79 - 512 out of 512 successfully perturbed
apgd-ce - 10/79 - 512 out of 512 successfully perturbed
apgd-ce - 11/79 - 512 out of 512 successfully perturbed
apgd-ce - 12/79 - 512 out of 512 successfully perturbed
apgd-ce - 13/79 - 512 out of 512 successfully perturbed
apgd-ce - 14/79 - 512 out of 512 successfully perturbed
apgd-ce - 15/79 - 512 out of 512 successfully perturbed
apgd-ce - 16/79 - 512 out of 512 successfully perturbed
apgd-ce - 17/79 - 512 o

In [8]:
clean_accuracy(model, x_adv, y, device=device, batch_size=128)

0.0009

In [9]:
#save images
count = 0
label_list = []
for a_i in x_adv:
    save_image(a_i, "../BLIP/experiment_1/imagenet/adv_test_standard_aa_Linf_eps_4_wrn50_2/{}.png".format(count))
    count +=1

In [ ]:
inputFile = "../BLIP/experiment_1/imagenet/clean_test_labels_50K.txt"

# Opening the given file in write mode
with open(inputFile, 'w') as filedata:

   # Traverse in each element of the input list 
   for item in y.cpu():
   
      # Writing each element of the list into the file
      # Here “%s\n” % syntax is used to move to the next line after adding an item to the file.
        filedata.write("%s\n" % item.numpy().item())